In [62]:
# import PyPDF2
import fitz
# import copy
import os
import re

In [3]:
file = "en_2022-2024 WAG COP.pdf"

In [5]:
def writeImage(image_path, image_bytes, count=1):
    # string = "page80_element_2.405_1.png"
    if count == 1:
        image_path = image_path.replace(".png", f"_{count}.png")
    else:
        image_path = re.sub(r"(\d?).png", count+".png", image_path)
    if os.path.exists(image_path):
        writeImage(image_path, image_bytes, count+1)
    else:
        with open(image_path, "wb") as image_file:
            image_file.write(image_bytes)

def getPageLayout(page):
    bbox = page.bleedbox
    if bbox.x1-bbox.y1 < 0:
        return "portrait"
    else:
        return "landscape"
    
def extract_text_and_images(pdf_path, output_folder, target_page, regex_pattern):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Check if the target page is within the range of pages
    if target_page < 0 or target_page >= len(pdf_document):
        print("Error: Invalid target page number.")
        return
    
    page = pdf_document.load_page(target_page)
    page_layout = getPageLayout(page)

    # Extract text from the page
    page_text = page.get_text()
    # print(page_text)
    found_text = re.findall(regex_pattern, page_text)
    print("FOUND", found_text)
    # Extract images and their positions
    images = page.get_images(full=True)
    
    # Loop through images on the page
    for img_info in images:
        xref = img_info[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Get the text closest and above the image
        bbox_image = page.get_image_bbox(img_info[7])
        # print("bbox img", bbox_image)

        if (bbox_image.bottom_right - bbox_image.top_left)[0] < 75:
            # assumed to be a 
            print("assuming", img_info[7], "is a judge symbol")
            continue


        found = ""
    
        optionA = []
        closestx = 1e6
        for item in found_text:
            for field in page.search_for(item):
                if page_layout == "portrait":
                    text_y1 = field.x1
                    text_x1 = abs(field.y0 - page.bleedbox.y1)
                else:
                    text_y1 = field.y1
                    text_x1 = field.x1
                x = round(bbox_image.y0 - round(text_y1))
                # print(field)
                # print(bbox_image)
                if x < 0:
                    continue
                if x < 15:
                    # In this case the number is too close to the picture and it is assumed its a number from the points; e.g. "3.20 P." instead of element 3.20
                    continue
                if x == closestx:
                    optionA.append(item)
                    closestx = x
                elif x < closestx:
                    optionA = [item]
                    closestx = x
                # print(optionA)
                for option in optionA:
                    # print(field)
                    # print(bbox_image.x0, text_x1, bbox_image.x1)
                    if bbox_image.x0 < text_x1 < bbox_image.x1:
                        # print("hurray", item)
                        found = option
                        pass
            
        if found != "":
            image_path = os.path.join(output_folder, f"page{target_page}_element_{found}.png")
            
            writeImage(image_path, image_bytes)
        else:
            print("warning! could not process ", img_info[7])
    pdf_document.close()

# Example usage
pdf_file_path = file
output_folder = "../data/images/"
target_page = 104  # Specify the page number you want to extract text and images from
regex_pattern = re.compile(r"(\d+\.\d+)\s?\n")  # Specify your regex pattern here
# extract_text_and_images(pdf_file_path, output_folder, target_page, regex_pattern)


In [98]:
pdf_document = fitz.open(file)
page = pdf_document.load_page(page_id=165)
getPageLayout(page=page)

'portrait'

In [99]:
page_text = page.get_text()
# print(page_text)
found_text = re.findall(r"(\d+\.\d+)\s+\n?(?!P\.)", page_text)
print(found_text)

['4.000', '4.103', '4.203', '4.303', '4.403', '4.503', '4.603', '4.104', '4.304', '4.404', '4.504', '4.604', '4.105', '4.205', '4.305', '4.405', '4.505', '4.605', '4.805']


In [126]:
# left, up, right, bottom
# clip = fitz.Rect(28.19099998474121 - 50, 113.76000213623047 - 50, 145.79100036621094 + 50, 186.24000549316406)
clip = page.get_image_bbox(page.get_images()[1][7])
# clip = fitz.Rect(168.59100341796875, 103.19999694824219 - 50, 277.0710144042969, 162.72000122070312)
pix = page.get_pixmap(matrix=fitz.Matrix(2, 2), clip=clip)
pix.save("test.png")

In [134]:
def findText(page, image, found_text, page_layout):
    found = ""
    optionA = []
    closestx = 1e6
    bbox_image = page.get_image_bbox(image[7])
    print("bbox", bbox_image)
    for item in found_text:
        for field in page.search_for(item):
            print(item, field)
            # some pages have a portrait mode, other landscape mode. 
            # This is important when matching the text with images. 
            # if the page is in portrait mode, we need to do a new calculation to make sure the matching works.
            if page_layout == "portrait":
                text_y1 = field.x1
                text_x1 = abs(field.y0 - page.bleedbox.y1)
                # text_x1 = field.y0
            elif page_layout == "landscape":
                text_y1 = field.y1
                text_x1 = field.x1
            elif page_layout == "other":
                text_y1 = field.y1
                text_x1 = abs( field.x1 - page.bleedbox.y1)
            diff = round(bbox_image.y0 - round(text_y1 /10)* 10)
            if diff < 0:
                continue
            if diff < 10:
                # In this case the number is too close to the picture and it is assumed its a number from the points; e.g. "3.20 P." instead of element 3.20
                print("yes")
                continue
            if diff == closestx:
                optionA.append([item, text_x1])
                closestx = diff
            elif diff < closestx:
                optionA = [[item, text_x1]]
                closestx = diff
    
    print(optionA)
    for option, text_x1 in optionA:
        print(option, bbox_image.x0, text_x1, bbox_image.x1)
        if bbox_image.x0 - 35 < text_x1 < bbox_image.x1 :
            found = option
            print("found", found)
   
    
    if found == "":
        distance = 1e6
        for option, text_x1 in optionA:
            print(option, bbox_image.x1 - text_x1 , distance)
            if 0 < bbox_image.x1 - text_x1 < distance:
                distance = bbox_image.x1 - text_x1
                found = option

    return found

findText(page, page.get_images()[1], found_text, "portrait" )

bbox Rect(686.510009765625, 295.67999267578125, 794.27001953125, 353.5199890136719)
4.000 Rect(35.74916076660156, 495.4164733886719, 46.94337844848633, 520.469482421875)
4.103 Rect(59.855918884277344, 793.6513671875, 68.77107238769531, 813.6295166015625)
4.203 Rect(59.855918884277344, 671.2512817382812, 68.77107238769531, 691.2293701171875)
4.303 Rect(59.855918884277344, 536.611572265625, 68.77107238769531, 556.589599609375)
4.403 Rect(59.855918884277344, 476.01507568359375, 68.77107238769531, 495.9931640625)
4.503 Rect(59.855918884277344, 422.0206298828125, 68.77107238769531, 441.99871826171875)
4.603 Rect(59.855918884277344, 273.52227783203125, 68.77107238769531, 293.5003662109375)
4.104 Rect(160.29592895507812, 793.6513671875, 169.21107482910156, 813.6295166015625)
4.304 Rect(160.29592895507812, 536.6099243164062, 169.21107482910156, 556.5880126953125)
4.404 Rect(160.29592895507812, 476.0093994140625, 169.21107482910156, 495.98748779296875)
4.504 Rect(160.29592895507812, 422.0093994

'4.605'

In [70]:
page.bleedbox.y1

595.3200073242188

In [58]:
round(397.6838073730469/10)* 10

400

In [57]:
round(396.72381591796875/10)* 10

400

In [60]:
page.get_textbox((358.2958984375, 129.9893798828125, 367.2110595703125, 147.74560546875))

'3.35 '